In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from tqdm.notebook import tqdm
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras import Model
tqdm.pandas()

     |████████████████████████████████| 2.6 MB 7.2 MB/s 
     |████████████████████████████████| 895 kB 63.9 MB/s 
     |████████████████████████████████| 636 kB 57.5 MB/s 
     |████████████████████████████████| 3.3 MB 54.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
data = pd.read_pickle("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/train_limited_brands.pkl",compression="zip")

In [4]:
data.dropna(inplace=True)

In [5]:
len(data)

2116687

In [6]:
# indexes = [i for i,j in dict(data.BROWSE_NODE_ID.value_counts()).items() if j<10000]
# len(indexes)
# all_groups = sorted(data.BROWSE_NODE_ID.value_counts().index)
# k = 1
# d = {}
# for i in all_groups:
#     if i in indexes:
#         d[i]=0
#     else:
#         d[i]=k
#         k+=1
# max(d.values())
# def replace(x):
#     return d[x]
# data["BROWSE_NODE_ID"] = data["BROWSE_NODE_ID"].progress_apply(replace)
# max(data["BROWSE_NODE_ID"].value_counts().index)

In [7]:
X = data.keywords.values
y = data.BROWSE_NODE_ID.values

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [9]:
import joblib
joblib.dump(le,"/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/niyati_mini_all.joblib")

['/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/niyati_mini_all.joblib']

In [10]:
len(le.classes_)

9643

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=14)

In [12]:
tokenizer = BertTokenizer.from_pretrained("google/bert_uncased_L-4_H-256_A-4")

In [13]:
class Generator(tf.keras.utils.Sequence) :
  
    def __init__(self, batch,labels, batch_size=64) :
        self.batch = batch
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self) :
        return self.batch.shape[0] // self.batch_size
  
    def __getitem__(self, idx) :
        batch_x = self.batch[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        batch_x = dict(tokenizer(list(batch_x),padding="max_length",truncation=True, max_length=128))

        batch_x = [
                   tf.convert_to_tensor(np.array(batch_x['input_ids'])),
                   tf.convert_to_tensor(np.array(batch_x['attention_mask']))
        ]
        return batch_x,tf.convert_to_tensor(batch_y)

In [14]:
rlr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto',
    min_delta=0, cooldown=0, min_lr=0.001,restore_best_weights=True)

es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience = 7,
    verbose=1, mode="auto", baseline=None, restore_best_weights=True,
)

nans = tf.keras.callbacks.TerminateOnNaN()

In [15]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)

In [16]:
def build_model(max_len=128):
    input1 = Input(shape=(max_len),dtype='int32')
    input2 = Input(shape=(max_len),dtype='int32')

    bert = TFBertForSequenceClassification.from_pretrained("google/bert_uncased_L-4_H-256_A-4",num_labels = len(le.classes_), output_hidden_states=True, from_pt=True)
    hidden = bert.bert([input1,input2]).hidden_states[-1]
    
    hidden = Dropout(0.3)(hidden)
    hidden = Dense(max_len//2,activation="tanh")(hidden)
    hidden = Dense(max_len//2,activation="tanh")(hidden)
    hidden = Flatten()(hidden)
    hidden = Dense(max_len*72,activation="tanh")(hidden)
    output = Dense(len(le.classes_),activation="softmax",)(hidden)
    model = Model(inputs=[input1,input2], outputs = output)
        
    print(model.summary())
    return model

In [17]:
training = Generator(X_train,y_train)
testing = Generator(X_test,y_test)
model = build_model()
model.compile(loss="sparse_categorical_crossentropy", optimizer="SGD",metrics=['accuracy'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 11170560    input_1[0][0]                    
                                                                 input_2[0][0]                    
_______________________________________________________

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
tf.keras.models.load_model('/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/niyati_mini_all2.h5')

In [20]:
model.fit(training, 
        epochs=1,   
        validation_data = testing,
        callbacks=[rlr,es,nans])

29765/29765 [==============================] - 6524s 219ms/step - loss: 2.1932 - accuracy: 0.5909 - val_loss: 1.6873 - val_accuracy: 0.6569


In [21]:
model.save("/content/drive/MyDrive/Amazon ML Challenge 2021/dataset/niyati_mini_all3.h5")